In [23]:
import numpy as np
import pandas as pd
import datetime
import sys, os

# Data parsing

In [24]:
# Parse html file
DAY = datetime.datetime.now().strftime("%d").lstrip('0')
MONTH = datetime.datetime.now().strftime("%m").lstrip('0')
# DAY = 23
# MONTH = 6
# SUB = '/scrf'
SUB = ''
file_name = 'Оперативные данные _ Коронавирус COVID–19_ Официальная информация о коронавирусе в России на портале – стопкоронавирус.рф.html'

# Save of "https://xn--80aesfpebagmfblc0a.xn--p1ai/information/" page
path = f'html_data/{DAY}-{MONTH}{SUB}/{file_name}'
parse_df = pd.read_html(path, encoding='utf-8')[0]

parse_df.head()

,Регион,Выявлено,Новые,Активные ? Число больных в настоящее время,Выздоровело,Умерло
0,Москва,217791,885,71928,142194,3669
1,Московская область,55853,467,26019,29002,832
2,Санкт-Петербург,23071,221,7010,15100,961
3,Нижегородская область,16414,217,6636,9566,212
4,Свердловская область,11684,192,5106,6496,82


In [25]:
# Modifi data frame
parse_df = parse_df.drop(parse_df.columns[2:4] , axis=1)
parse_df.columns = ['Region/City', 'Confirmed', 'Recovered', 'Deaths']
parse_df['Date'] = pd.Timestamp(f'2020-{MONTH}-{DAY}')
parse_df

,Region/City,Confirmed,Recovered,Deaths,Date
0,Москва,217791,142194,3669,2020-06-25
1,Московская область,55853,29002,832,2020-06-25
2,Санкт-Петербург,23071,15100,961,2020-06-25
3,Нижегородская область,16414,9566,212,2020-06-25
4,Свердловская область,11684,6496,82,2020-06-25
...,...,...,...,...,...
80,Республика Алтай,511,215,0,2020-06-25
81,Еврейская автономная область,408,332,5,2020-06-25
82,Севастополь,215,170,3,2020-06-25
83,Чукотский автономный округ,135,122,1,2020-06-25


# Preparing parsed data

In [26]:
file_name = 'release/covid19-russia-cases-scrf.csv'
rus_df = pd.read_csv(file_name)
rus_df.tail()

,Date,Region/City,Region/City-Eng,Region_ID,Day-Confirmed,Day-Deaths,Day-Recovered,Confirmed,Deaths,Recovered
8928,2020-06-24,Челябинская область,Chelyabinsk region,74.0,185.0,0.0,328.0,6268.0,70.0,3343.0
8929,2020-06-24,Чеченская Республика,Chechen Republic,95.0,11.0,0.0,1.0,1645.0,20.0,1175.0
8930,2020-06-24,Чукотский автономный округ,Chukotka Autonomous Okrug,87.0,1.0,0.0,11.0,135.0,1.0,122.0
8931,2020-06-24,Ямало-Ненецкий АО,Yamalo-Nenets Autonomous Okrug,89.0,137.0,3.0,63.0,4553.0,33.0,1967.0
8932,2020-06-24,Ярославская область,Yaroslavl region,76.0,44.0,1.0,178.0,4526.0,18.0,2864.0


In [27]:
# Add day columns
parse_df['Day-Confirmed'] = 0
parse_df['Day-Deaths'] = 0
parse_df['Day-Recovered'] = 0

# Strip text data
rus_df['Region/City'] = rus_df['Region/City'].astype('str').str.strip('\u200b') 
parse_df['Region/City'] = parse_df['Region/City'].astype('str').str.strip('\u200b')

# Rename regions
rename_dict = {
    'Ямало-Ненецкий автономный округ' : 'Ямало-Ненецкий АО',
    'Республика Северная Осетия — Алания' : 'Республика Северная Осетия - Алания',
}

parse_df['Region/City'] = parse_df['Region/City'].replace(rename_dict)

In [28]:
def upd(row):
    reg = row['Region/City']
    
    row['Day-Confirmed'] = row['Confirmed'] - rus_df[rus_df['Region/City'] == reg]['Day-Confirmed'].sum()
    row['Day-Deaths']    = row['Deaths'] - rus_df[rus_df['Region/City'] == reg]['Day-Deaths'].sum()
    row['Day-Recovered'] = row['Recovered'] - rus_df[rus_df['Region/City'] == reg]['Day-Recovered'].sum()
    
    row['Day-Confirmed'] = row['Confirmed'] if np.isnan(row['Day-Confirmed']) else row['Day-Confirmed']
    row['Day-Deaths']    = row['Deaths'] if np.isnan(row['Day-Deaths']) else row['Day-Deaths']
    row['Day-Recovered'] = row['Recovered'] if np.isnan(row['Day-Recovered']) else row['Day-Recovered']
    
    return row.drop('Region/City')

parse_df = parse_df.groupby('Region/City').apply(lambda df: upd(df.iloc[0])).reset_index()

In [29]:
# Check for missed regions
parse_df[np.isnan(parse_df['Day-Confirmed'])]

,Region/City,Confirmed,Recovered,Deaths,Date,Day-Confirmed,Day-Deaths,Day-Recovered


# Data checking

In [30]:
print('Russia Confirmed:', parse_df.groupby('Region/City')['Confirmed'].max().sum(),
      'Day-Confirmed:', parse_df.groupby('Region/City')['Day-Confirmed'].sum().sum())
print('Russia Deaths:', parse_df.groupby('Region/City')['Deaths'].max().sum(),
      'Day-Deaths:', parse_df.groupby('Region/City')['Day-Deaths'].sum().sum())
print('Russia Recovered:', parse_df.groupby('Region/City')['Recovered'].max().sum(),
      'Day-Recovered:', parse_df.groupby('Region/City')['Day-Recovered'].sum().sum())

# rus_regs = rus_sum.groupby('Region/City')['Recovered'].max().reset_index()
# rus_regs['Recovered-ByDay'] = rus_sum.groupby('Region/City')['Day-Recovered'].sum().reset_index(drop=True)
# rus_regs.loc[rus_regs['Recovered'] != rus_regs['Recovered-ByDay']]
# # rus_regs.to_csv('rus_regs.csv')

Russia Confirmed: 613994 Day-Confirmed: 7113.0
Russia Deaths: 8605 Day-Deaths: 92.0
Russia Recovered: 375164 Day-Recovered: 6342.0


# Data saving

In [31]:
# Save full update table
parse_df.to_csv(f'covid19-russia-cases-upd-full-scrf.csv', index=False)

In [32]:
# Drop rows without day changes
parse_df = parse_df[(parse_df['Day-Confirmed'] != 0) |
                (parse_df['Day-Deaths'] != 0) |
                (parse_df['Day-Recovered'] != 0)]

In [33]:
parse_df.to_csv(f'old_data/covid19-russia-cases-upd{DAY}-{MONTH}-scrf.csv', index=False)
parse_df.to_csv('covid19-russia-cases-upd-scrf.csv', index=False)